In [1]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split 

Download the Data from https://www.kaggle.com/datasets/rmisra/news-category-dataset

In [2]:
path = ""
workdata_path = "../working_data/"

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rmisra/news-category-dataset")

print("Path to dataset files:", path)

/home/y41000/repos/bert-vs-rag/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 26.5M/26.5M [00:02<00:00, 12.3MB/s]

Extracting files...


Path to dataset files: /home/y41000/.cache/kagglehub/datasets/rmisra/news-category-dataset/versions/3


In [4]:
# Copy the data to our wor diratory
# !cp  /Users/yhonain/.cache/kagglehub/datasets/rmisra/news-category-dataset/versions/3/News_Category_Dataset_v3.json /Users/yhonain/nlp_task/data/

In [5]:
data = pd.read_json(f'{path}/News_Category_Dataset_v3.json', lines = True)

In [6]:
data.shape

(209527, 6)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209527 entries, 0 to 209526
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   link               209527 non-null  object        
 1   headline           209527 non-null  object        
 2   category           209527 non-null  object        
 3   short_description  209527 non-null  object        
 4   authors            209527 non-null  object        
 5   date               209527 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 9.6+ MB


In [8]:
pd.DataFrame(data['category'].value_counts()).join(
    pd.DataFrame(data['category'].value_counts(normalize=True))
)

,count,proportion
category,,
POLITICS,35602,0.169916
WELLNESS,17945,0.085645
ENTERTAINMENT,17362,0.082863
TRAVEL,9900,0.047249
STYLE & BEAUTY,9814,0.046839
PARENTING,8791,0.041956
HEALTHY LIVING,6694,0.031948
QUEER VOICES,6347,0.030292
FOOD & DRINK,6340,0.030259


In [9]:
pd.crosstab(data[data['authors'].isin(list(data['authors'].value_counts()[:20].index))]['category'], data[data['authors'].isin(list(data['authors'].value_counts()[:20].index))]['authors'])

authors,,Andy McDonald,Bill Bradley,Carly Ledbetter,Caroline Bologna,Cole Delbyck,Curtis M. Wong,Dana Oliver,David Moye,Dominique Mosbergen,Ed Mazza,Igor Bobic,Julia Brucculieri,Lee Moran,Mary Papenfuss,Michelle Manetti,Nina Golgowski,"Reuters, Reuters",Ron Dicker,Sam Levine
category,,,,,,,,,,,,,,,,,,,,
ARTS,19,0,0,0,0,0,7,0,0,4,0,0,0,0,0,0,0,0,1,0
ARTS & CULTURE,32,4,0,1,1,3,6,0,6,3,6,0,5,41,6,0,7,0,3,0
BLACK VOICES,1008,14,0,1,0,0,4,2,53,6,12,0,0,31,7,0,35,23,40,1
BUSINESS,991,0,0,0,1,0,0,0,2,21,21,1,0,2,25,0,13,294,13,2
COLLEGE,111,0,0,0,0,0,0,0,0,3,2,0,0,0,0,0,2,1,1,0
COMEDY,729,857,167,4,0,9,0,0,59,25,343,0,15,841,129,0,8,0,414,0
CRIME,977,0,0,0,0,0,0,0,97,31,58,1,0,92,76,0,251,45,45,0
CULTURE & ARTS,370,0,0,0,0,0,1,0,0,1,0,0,0,2,0,0,0,1,0,0
DIVORCE,1731,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0,0


In [10]:

perc = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.99]
data['news'] = data['headline'] +' -- '+ data['short_description']
pd.concat([
    pd.DataFrame(data['headline'].str.len().describe(percentiles=perc)),
    pd.DataFrame(data['short_description'].str.len().describe(percentiles=perc)),
    pd.DataFrame(data['news'].str.len().describe(percentiles=perc))
], axis=1)

,headline,short_description,news
count,209527.000000,209527.000000,209527.000000
mean,58.415355,114.208670,176.624025
std,18.808506,80.840575,78.552970
min,0.000000,0.000000,4.000000
10%,33.000000,9.000000,79.000000
20%,42.000000,46.000000,112.000000
30%,49.000000,70.000000,137.800000
40%,55.000000,94.000000,158.000000
50%,60.000000,120.000000,174.000000
60%,64.000000,122.000000,187.000000


# DATA PROCESSING

In [11]:
data_work = data[['news', 'category']]


In [12]:
rest, validation = train_test_split(data_work,test_size=0.3 ,random_state=2025, stratify =data['category'])

In [13]:
validation.to_excel(f'{workdata_path}validation.xlsx', index=False)
validation.shape


(62859, 2)

In [14]:
proportions = [0.001, 0.005, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
for p in proportions:
    _, v = train_test_split(rest,test_size=p,random_state=2025 ,stratify =rest['category'])
    v.to_excel(f'{workdata_path}prop_{p}.xlsx', index=False)
rest.to_excel(f'{workdata_path}prop_1.xlsx', index=False)

In [58]:
data_work

,headline,short_description,category
0,Over 4 Million Americans Roll Up Sleeves For O...,Health experts said it is too early to predict...,U.S. NEWS
1,"American Airlines Flyer Charged, Banned For Li...",He was subdued by passengers and crew when he ...,U.S. NEWS
2,23 Of The Funniest Tweets About Cats And Dogs ...,"""Until you have a dog you don't understand wha...",COMEDY
3,The Funniest Tweets From Parents This Week (Se...,"""Accidentally put grown-up toothpaste on my to...",PARENTING
4,Woman Who Called Cops On Black Bird-Watcher Lo...,Amy Cooper accused investment firm Franklin Te...,U.S. NEWS
...,...,...,...
209522,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,Verizon Wireless and AT&T are already promotin...,TECH
209523,Maria Sharapova Stunned By Victoria Azarenka I...,"Afterward, Azarenka, more effusive with the pr...",SPORTS
209524,"Giants Over Patriots, Jets Over Colts Among M...","Leading up to Super Bowl XLVI, the most talked...",SPORTS
209525,Aldon Smith Arrested: 49ers Linebacker Busted ...,CORRECTION: An earlier version of this story i...,SPORTS
